# Projektaufgabe - Lazy-PRM-Erweiterung und Kollisionstests
Modul: Roboterprogrammierung im Studiengang Robotik und KI Master  
Professor: Prof. Dr. Björn Hein  
Semester: Wintersemester 2025/26  
Gruppe: Planlos im Arbeitsraum (Analena Krieger, Julia Kramme & Melina Schwan)  
Abgabe: XXX 

## Aufgabe 1: Vergleich Lazy-PRM vs. Node-Checked-PRM
In diesem Notebookabschnitt untersuchen wir den Einfluss der Anzahl der Kollisionsüberprüfungen beim Vergleich zweier Varianten von PRM:

1. **Lazy-PRM (gegeben)**  
   - Knoten und Kanten werden beim Erstellen **nicht** auf Kollision geprüft  
   - Eine Kollisionsprüfung erfolgt erst, wenn ein potenzieller Pfad gefunden wurde  

2. **Node-Checked-PRM (neu implementiert)**  
   - Jeder erzeugte Knoten wird **sofort** auf Kollision geprüft  
   - Kanten werden weiterhin erst beim Pfadtest überprüft  

Unser Ziel ist es, diese beiden Varianten anhand mehrerer Benchmark-Aufgaben systematisch zu vergleichen und dabei folgende Größen zu erfassen:

- Anzahl der Kollisionsprüfungen  
- Gesamtplanungszeit  
- Länge des gefundenen Pfads  
- Größe der Roadmap (Anzahl der Knoten und Kanten)

Die Ergebnisse werden von uns statistisch ausgewertet, visualisiert und anschließend interpretiert.


In [ ]:
# Imports & Testumgeubung
import pandas
import numpy as np
import matplotlib.pyplot as plt
import time
from statistics import mean
import IPTestSuite as ts
from IPLazyPRM import LazyPRM
from IPEnvironment import CollisionChecker

In [ ]:
# Variante b - Knoten sofort prüfen
import networkx as nx
from scipy.spatial import cKDTree

class NodeCheckedPRM(LazyPRM):
    """
    Aufgabe 1b:
    - Jeder erzeugte Knoten wird sofort auf Kollision geprüft.
    - Kanten bleiben ungeprüft wie bei Lazy-PRM.
    """

    def _buildRoadmap(self, numNodes, kNearest):
        addedNodes = []

        # 1) Knoten erzeugen & direkt prüfen
        for i in range(numNodes):
            pos = self._getRandomPosition()

            if self._collisionChecker.pointInCollision(pos):
                continue  # kollidierender Knoten -> nicht hinzufügen

            self.graph.add_node(self.lastGeneratedNodeNumber, pos=pos)
            addedNodes.append(self.lastGeneratedNodeNumber)
            self.lastGeneratedNodeNumber += 1

        # 2) Kanten verbinden (kein Kollisionscheck, lazy)
        posList = list(nx.get_node_attributes(self.graph, 'pos').values())
        kdTree = cKDTree(posList)

        for node in addedNodes:
            result = kdTree.query(self.graph.nodes[node]["pos"], k=kNearest)

            for idx in result[1]:
                c_node = [n for n, d in self.graph.nodes(data=True)
                          if d["pos"] == posList[idx]][0]
                if node != c_node:
                    self.graph.add_edge(node, c_node)


In [ ]:
# Benchmarks-Umgebungen
benchmarks = [
    ts.benchList[0],  # Trap
    ts.benchList[1],  # Bottleneck
    ts.benchList[3],  # MyField
]

In [ ]:
# Konfig Parameter und Hindernisumgebungen anzeigen
from IPVISLazyPRM import lazyPRMVisualize
from IPLazyPRM import LazyPRM

config = {
    "initialRoadmapSize": 150,   # zufällige Knotenanzahl
    "updateRoadmapSize": 50,     # neue Knoten wenn kein Pfad gefunden wurde
    "kNearest": 10,              # Anzahl Nachbarknoten
    "maxIterations": 80          # maximale Anzahl Verbesserungsiterationen
}

# Benchmark-Liste: Trap + Bottleneck + MyField
benchmarks = [ts.benchList[0], ts.benchList[1], ts.benchList[3]]

for benchmark in benchmarks:
    print("Benchmark:", benchmark.name)

    # 1) Lazy-PRM Visualisierung
    planner = LazyPRM(benchmark.collisionChecker)
    solution = planner.planPath(benchmark.startList, benchmark.goalList, config)

    fig = plt.figure(figsize=(8,8))
    fig.suptitle(f"Lazy-PRM – {benchmark.name}", fontsize=16)
    ax = fig.add_subplot(111)

    lazyPRMVisualize(planner, solution, ax=ax)
    plt.show()

    # 2) Node-Checked-PRM Visualisierung
    planner2 = NodeCheckedPRM(benchmark.collisionChecker)
    solution2 = planner2.planPath(benchmark.startList, benchmark.goalList, config)

    fig = plt.figure(figsize=(8,8))
    fig.suptitle(f"Node-Checked-PRM – {benchmark.name}", fontsize=16)
    ax = fig.add_subplot(111)

    lazyPRMVisualize(planner2, solution2, ax=ax)
    plt.show()


In [ ]:
# Pfadplaner mehrfach auf demselben Benchmark ausführen und Parameter messen
def run_benchmark(planner_class, benchmark, config, runs=20):
    times = []
    collisions = []
    lengths = []
    sizes = []  # Roadmap-Größe

    for _ in range(runs):
        planner = planner_class(benchmark.collisionChecker)
        benchmark.collisionChecker.counter = 0

        # Zeit messen
        t0 = time.time()
        path = planner.planPath(benchmark.startList, benchmark.goalList, config)
        t1 = time.time()

        times.append(t1 - t0)
        collisions.append(benchmark.collisionChecker.counter)
        sizes.append(len(planner.graph.nodes()))   # Roadmapgröße messen

        # Pfadlänge
        if path != []:
            pts = [planner.graph.nodes[p]['pos'] for p in path]
            length = sum(np.linalg.norm(np.array(pts[j]) - np.array(pts[j+1]))
                         for j in range(len(pts)-1))
            lengths.append(length)      # Euklidische Pfadlänge im C-Space
        else:
            lengths.append(np.nan)

    return times, collisions, lengths, sizes        # Rückgabewerte

In [ ]:
# Lazy und Node-Checked Planer auf mehreren Benchmarks ausführen und Ergbenisse sammeln, sowie speichern
benchmarks = [ts.benchList[0], ts.benchList[1], ts.benchList[3]]  # Trap + Bottleneck + MyField

results = {}

for bench in benchmarks:
    print("Running:", bench.name)

    lazy = run_benchmark(LazyPRM, bench, config)
    node = run_benchmark(NodeCheckedPRM, bench, config)

    results[bench.name] = {
        "lazy": lazy,
        "node": node
    }


In [ ]:
## Aufgabe 1a: Vergleich Lazy-PRM vs. Node-Checked-PRM (Punktroboter)
# Ziel: Einfluss der Kollisionsprüfung bei einfachen Robotern messen.

# 1. CONFIG
# Punktroboter sind einfach, wir brauchen nicht extrem viele Knoten
config_point = {
    "initialRoadmapSize": 150,   
    "updateRoadmapSize": 50,     
    "kNearest": 10,              
    "maxIterations": 80          
}

# 2. BENCHMARKS AUSWÄHLEN
# Wir nutzen die Standard-Benchmarks aus der IPTestSuite
# Index 0 = Trap (Falle)
# Index 1 = Bottleneck (Engstelle)
benchmarks_1a = [ts.benchList[0], ts.benchList[1]]

# 3. VISUALISIERUNG
print("Visualisiere Benchmarks für Aufgabe 1a...")

fig, axs = plt.subplots(1, 2, figsize=(14, 7))

for i, bench in enumerate(benchmarks_1a):
    ax = axs[i]
    ax.set_title(f"Benchmark: {bench.name}")
    
    # Planer ausführen (Lazy Variante)
    planner = LazyPRM(bench.collisionChecker)
    sol = planner.planPath(bench.startList, bench.goalList, config_point)
    
    # Visualisieren
    lazyPRMVisualize(planner, sol, ax=ax)

plt.tight_layout()
plt.show()

# 4. STATISTISCHE AUSWERTUNG (30 Runs)
print("\n--- Starte statistische Auswertung (Aufgabe 1a) ---")
# Hier nutzen wir die gleiche Funktion wie in 1b und 1c
# Sie vergleicht Lazy-PRM (Orange) mit Node-Checked-PRM (Blau/Lila)
evaluate_and_plot(benchmarks_1a, config_point, title_prefix="Aufgabe 1a", runs=30)


## Fazit von Aufgabe 1a  
- Lazy-PRM braucht im Mittel weniger Kollisionsprüfungen und ist dadurch schneller
- Node-Checked-PRM erzeugt meist stabilere Roadmaps (zB weniger ungültige Kanten) und teilweise kürzere Pfade

## Vorbereitungen für die Aufgabe 1b und 1c

In [ ]:
# NEUE IMPORTS FÜR AUFGABE 1b und 1c
import math
import numpy as np
from shapely.geometry import Point, Polygon, LineString

try:
    from IPEnvironmentShapeRobot import CollisionCheckerShapeRobot, ShapeRobotWithOrientation
    from IPVISBasicPRM import basicPRMVisualizeWspace # Für Visualisierung mit Rotation
except ImportError as e:
    print(f"Hinweis: Ein Import fehlte ({e}). Bitte sicherstellen, dass die Dateien im Ordner liegen.")

# HILFSFUNKTION FÜR STATISTIK
def evaluate_and_plot(benchmarks, config, title_prefix="", runs=30):
    """
    Führt Lazy-PRM und Node-Checked-PRM mehrfach aus und plottet die Ergebnisse.
    """
    results = {}
    
    # 1. Daten sammeln
    for bench in benchmarks:
        print(f"Starte Benchmark '{bench.name}' ({runs} Durchläufe)...")
        lazy_data = run_benchmark(LazyPRM, bench, config, runs=runs)
        node_data = run_benchmark(NodeCheckedPRM, bench, config, runs=runs)
        
        results[bench.name] = {"lazy": lazy_data, "node": node_data}

    # 2. Plotten
    for bench_name, data in results.items():
        # Mittelwerte (Time, Collisions, PathLen, Size)
        # Index 0=Time, 1=Colls, 2=Len, 3=Size 
        means_lazy = [np.nanmean(data["lazy"][i]) for i in range(4)]
        means_node = [np.nanmean(data["node"][i]) for i in range(4)]
        
        fig, axs = plt.subplots(1, 4, figsize=(16, 4))
        fig.suptitle(f"{title_prefix}: {bench_name}", fontsize=14)
        
        titles = ["Planungszeit (s)", "Kollisionschecks", "Pfadlänge", "Roadmap Größe"]
        colors = ['gold', 'tomato', 'cornflowerblue', 'mediumpurple']
        
        for i in range(4):
            ax = axs[i]
            x_pos = [0, 1]
            vals = [means_lazy[i], means_node[i]]
            
            bars = ax.bar(["Lazy", "NodeCheck"], vals, color=colors[i], alpha=0.8)
            ax.set_title(titles[i])
            ax.grid(axis='y', alpha=0.3)
            
            # Kennzeichnung falls kein Pfad gefunden wurde (NaN)
            if np.isnan(vals[0]): ax.text(0, 0, "X", color='red', ha='center', fontsize=20)
            if np.isnan(vals[1]): ax.text(1, 0, "X", color='red', ha='center', fontsize=20)

        plt.tight_layout()
        plt.show()

## Aufgabe 1b: Vergleich mit 3-DoF Shape-Robotern

In diesem Abschnitt vergleichen wir die Varianten anhand von Robotern, die eine Ausdehnung und Orientierung besitzen. 
Der Konfigurationsraum erweitert sich auf $(x, y, \theta)$.
Die Kollisionsprüfung ist hier deutlich teurer als bei Punktrobotern, weshalb wir erwarten, dass **Lazy-PRM** hier seine Stärken in der Laufzeit ausspielen kann, da es teure Checks auf Kanten vermeidet, die gar nicht Teil des Pfades sind.

**Gewählte Benchmarks:**
1. Shape-Roboter A (z.B. L-Shape)
2. Shape-Roboter B (z.B. Rechteck/Triangle in enger Umgebung)

In [ ]:
## Aufgabe 1b: Vergleich mit Shape-Robotern (3-DoF) 

# IMPORTS
import numpy as np
import math
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString

# Sicherstellen, dass die Klassen da sind
try:
    from IPEnvironmentShapeRobot import CollisionCheckerShapeRobot, ShapeRobotWithOrientation
except ImportError:
    pass

# 1. Hilfsklasse
class CustomBenchmark:
    def __init__(self, name, cc, start, goal):
        self.name = name
        self.collisionChecker = cc
        self.startList = start
        self.goalList = goal

# ROBOTER DEFINIEREN
# L-Form (zentriert)
poly_L = Polygon([(0,0), (2.0,0), (2.0,0.5), (0.5,0.5), (0.5,2.5), (0,2.5)])
poly_L = Polygon([(x - poly_L.centroid.x, y - poly_L.centroid.y) for x,y in poly_L.exterior.coords])

# Dreieck (zentriert)
poly_Tri = Polygon([(-0.5, -0.5), (3.0, 0), (-0.5, 0.5)])
poly_Tri = Polygon([(x - poly_Tri.centroid.x, y - poly_Tri.centroid.y) for x,y in poly_Tri.exterior.coords])

limits_3dof = [[0.0, 22.0], [0.0, 22.0], [-math.pi, math.pi]]


# UMGEBUNG 1: LINIEN MIT SCHUTZZONEN
start_pos_lines = [2.0, 2.0]
goal_pos_lines  = [20.0, 20.0]

safe_zone_start = Point(start_pos_lines).buffer(3.5)
safe_zone_goal  = Point(goal_pos_lines).buffer(3.5)

env_lines = {}
np.random.seed(42) 
counter = 0
target_lines = 20 

while len(env_lines) < target_lines:
    p1 = np.random.uniform(0, 22, 2)
    angle = np.random.uniform(0, math.pi)
    length = np.random.uniform(2.0, 5.0)
    p2 = p1 + np.array([np.cos(angle), np.sin(angle)]) * length
    
    candidate_line = LineString([p1, p2]).buffer(0.15)
    
    if candidate_line.intersects(safe_zone_start) or candidate_line.intersects(safe_zone_goal):
        continue 
        
    env_lines[f"line_{counter}"] = candidate_line
    counter += 1

print(f"Umgebung 'Linien': {len(env_lines)} Hindernisse generiert.")


# UMGEBUNG 2: KREISE
env_circles = {
    "c1": Point(7, 11).buffer(2.5),
    "c2": Point(15, 11).buffer(2.5)
}


# BENCHMARKS ZUSAMMENBAUEN
benchmarks_1b = []

# 1. L-Shape in Linien
cc_L_lines = CollisionCheckerShapeRobot(env_lines, ShapeRobotWithOrientation(poly_L, limits=limits_3dof))
benchmarks_1b.append(CustomBenchmark("L-Shape (Lines)", cc_L_lines, [start_pos_lines + [0.0]], [goal_pos_lines + [1.57]]))

# 2. Triangle in Linien
cc_Tri_lines = CollisionCheckerShapeRobot(env_lines, ShapeRobotWithOrientation(poly_Tri, limits=limits_3dof))
benchmarks_1b.append(CustomBenchmark("Triangle (Lines)", cc_Tri_lines, [start_pos_lines + [0.0]], [goal_pos_lines + [0.0]]))

# 3. L-Shape in Kreisen
cc_L_circles = CollisionCheckerShapeRobot(env_circles, ShapeRobotWithOrientation(poly_L, limits=limits_3dof))
benchmarks_1b.append(CustomBenchmark("L-Shape (Circles)", cc_L_circles, [[1.0, 11.0, 0.0]], [[21.0, 11.0, 1.57]]))

# 4. Triangle in Kreisen
cc_Tri_circles = CollisionCheckerShapeRobot(env_circles, ShapeRobotWithOrientation(poly_Tri, limits=limits_3dof))
benchmarks_1b.append(CustomBenchmark("Triangle (Circles)", cc_Tri_circles, [[1.0, 11.0, 0.0]], [[21.0, 11.0, 0.0]]))


# VISUALISIERUNG
config = {
    "initialRoadmapSize": 150,
    "updateRoadmapSize": 50,
    "kNearest": 10,
    "maxIterations": 80
}

config_shape = config.copy()
config_shape["initialRoadmapSize"] = 250
config_shape["kNearest"] = 15
config_shape["maxIterations"] = 60

print("\n--- Visualisierung der 4 Szenarien ---")

fig, axs = plt.subplots(2, 2, figsize=(14, 14))
axs = axs.flatten() 

for i, bench in enumerate(benchmarks_1b):
    print(f"Plane für: {bench.name}...")
    try:
        planner = LazyPRM(bench.collisionChecker)
        sol = planner.planPath(bench.startList, bench.goalList, config_shape)
        
        ax = axs[i]
        
        if "Lines" in bench.name:
            x_s, y_s = safe_zone_start.exterior.xy
            x_g, y_g = safe_zone_goal.exterior.xy
            ax.plot(x_s, y_s, color='green', linestyle='--', alpha=0.3, label='Safe Zone')
            ax.plot(x_g, y_g, color='green', linestyle='--', alpha=0.3)
        
        try:
            basicPRMVisualizeWspace(planner, sol, ax=ax)
        except NameError:
            lazyPRMVisualize(planner, sol, ax=ax)
            
        ax.set_title(bench.name)
        
        if not sol:
            ax.text(0.5, 0.5, "KEIN PFAD", transform=ax.transAxes, color='red', fontsize=20, ha='center')

    except Exception as e:
        print(f"Fehler bei {bench.name}: {e}")

plt.tight_layout()
plt.show()

# STATISTIK
print("\n--- Starte statistische Auswertung ---")
# Hinweis: Falls 'evaluate_and_plot' fehlt, Block 1 ausführen!
evaluate_and_plot(benchmarks_1b, config_shape, title_prefix="Aufgabe 1b", runs=30)

## Aufgabe 1c: Vergleich mit Planar Manipulatoren

Hier betrachten wir kinematische Ketten (Roboterarme).
- **Benchmark 1:** 2-DoF Manipulator (Leicht)
- **Benchmark 2:** 3-DoF Manipulator (Schwer / Engstellen)

Da sich der Roboter im Gelenkwinkelraum bewegt, sind die Kollisionsberechnungen (Vorwärtskinematik pro Check) rechenintensiv. Wir untersuchen, ob die Strategie "Erst prüfen, dann verbinden" (Node-Checked) oder "Erst verbinden, dann prüfen" (Lazy) effizienter ist.

In [ ]:
import sys
print(sys.executable)


In [ ]:
import sympy as sp
sp.__version__


In [ ]:
# IMPORTS & KLASSEN FÜR AUFGABE 1c (aus dem Vorlesungs-Notebook)
#import copy
#from shapely.geometry import LineString, Polygon
#from shapely import plotting
#try:
#    from IPPlanarManipulator import PlanarRobot # Diese Datei muss im Ordner liegen
#except ImportError:
#    print("Fehler: 'IPPlanarManipulator.py' nicht gefunden.")

import copy
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Polygon
from shapely import plotting

from IPPlanarManipulator import PlanarRobot 
from IPEnvironment import CollisionChecker
from IPBenchmark import Benchmark

# 1. Kollisions-Checker für kinematische Ketten (Original aus deinem Notebook)
class KinChainCollisionChecker(CollisionChecker):
    def __init__(self, kin_chain, scene, limits=[[-3.0, 3.0], [-3.0, 3.0]], statistic=None, fk_resolution=0.1):
        super(KinChainCollisionChecker, self).__init__(scene, limits, statistic)
        self.kin_chain = kin_chain
        self.fk_resolution = fk_resolution
        self.dim = self.kin_chain.dim

    def getDim(self):
        return self.dim
        
    def pointInCollision(self, pos):
        # Bewegt den Roboter an die Konfiguration 'pos' (Winkel)
        self.kin_chain.move(pos)
        joint_positions = self.kin_chain.get_transforms()
        
        # Zähler für Statistik erhöhen (WICHTIG für Aufgabe 1)
        self.counter += 1 

        # Prüfe jedes Segment (Arm-Glied) auf Kollision
        for i in range(1, len(joint_positions)):
            if self.segmentInCollision(joint_positions[i-1], joint_positions[i]):
                return True
        return False
    
    def lineInCollision(self, startPos, endPos):
        # Interpoliert zwischen zwei Winkel-Konfigurationen
        steps = self.interpolate_line(startPos, endPos, self.fk_resolution)
        for pos in steps:
            if self.pointInCollision(pos):
                return True
        return False
    
    def segmentInCollision(self, startPos, endPos):
        # Prüft ein Liniensegment im Arbeitsraum gegen Hindernisse
        line_geom = LineString([(startPos[0], startPos[1]), (endPos[0], endPos[1])])
        for key, value in self.scene.items():
            if value.intersects(line_geom):
                return True
        return False
        
    def interpolate_line(self, startPos, endPos, step_l):
        # Hilfsfunktion für Interpolation im C-Space
        steps = []
        line = np.array(endPos) - np.array(startPos)
        line_l = np.linalg.norm(line)
        if line_l == 0: return [startPos]
        
        step = line / line_l * step_l
        n_steps = np.floor(line_l / step_l).astype(np.int32)
        c_step = np.array(startPos)
        for i in range(n_steps):
            steps.append(copy.deepcopy(c_step))
            c_step += step
        if not (c_step == np.array(endPos)).all():
            steps.append(np.array(endPos))
        return steps

    def drawObstacles(self, ax, inWorkspace=False):
        if inWorkspace:
            for key, value in self.scene.items():
                plotting.plot_polygon(value, add_points=False, color='red', alpha=0.5, ax=ax)

# 2. Visualisierungs-Helper (Roboter zeichnen)
def planarRobotVisualize(kin_chain, ax):
    joint_positions = kin_chain.get_transforms()
    # Zeichne Segmente
    for i in range(1, len(joint_positions)):
        xs = [joint_positions[i-1][0], joint_positions[i][0]]
        ys = [joint_positions[i-1][1], joint_positions[i][1]]
        ax.plot(xs, ys, color='blue', linewidth=3, alpha=0.7)
    # Zeichne Gelenke
    jx = [p[0] for p in joint_positions]
    jy = [p[1] for p in joint_positions]
    ax.scatter(jx, jy, color='black', zorder=5)

In [ ]:
# 1. SETUP ROBOTER
try:
    robot = PlanarRobot(n_joints=3) 
except:
    print("Fallback auf 2 Gelenke")
    robot = PlanarRobot(n_joints=2)

# Limits für 3D-Konfigurationsraum [-pi, pi]
c_limits = [[-np.pi, np.pi]] * robot.dim


# 2. BENCHMARK A: ENGSTELLE (Narrow Passage)
# Wand bei x=1.5 mit Lücke zwischen y=2 und y=3

obs_gap = {
    "wall_bottom": LineString([(1.5, -2), (1.5, 2)]).buffer(0.3),  # Wand unten
    "wall_top":    LineString([(1.5, 3), (1.5, 7)]).buffer(0.3)    # Wand oben
}

cc_gap = KinChainCollisionChecker(robot, obs_gap, limits=c_limits, fk_resolution=0.1)

# Start: Links (pi) -> Ziel: Oben (pi/2)
# Der Roboter muss von links nach oben schwenken, ohne die Wand zu treffen.
start_gap = [np.pi, 0.0, 0.0] 
goal_gap  = [np.pi/2, 0.0, 0.0]

#bench_gap = CustomBenchmark("1. Narrow Passage", cc_gap, [start_gap], [goal_gap])
bench_gap = Benchmark(
    name="1. Narrow Passage",
    collisionChecker=cc_gap,
    startList=[start_gap],
    goalList=[goal_gap],
    description="3-DOF planar arm must pass through a narrow gap",
    level=3
)


# 3. BENCHMARK B: HINDERNIS UMFAHREN (Obstacle Avoidance)
# Ein Block liegt im Weg auf der X-Achse.
# Start: Oben (pi/2) -> Ziel: Unten (-pi/2)

obs_block = {
    # Ein Rechteckiges Hindernis bei x=2.0, y=0.0
    "block": Polygon([(1.5, -1.0), (2.5, -1.0), (2.5, 1.0), (1.5, 1.0)])
}

cc_block = KinChainCollisionChecker(robot, obs_block, limits=c_limits, fk_resolution=0.1)

# Start: Senkrecht nach OBEN
start_block = [np.pi/2, 0.1, 0.1] 
# Ziel: Senkrecht nach UNTEN
goal_block  = [-np.pi/2, 0.1, 0.1]

# Validierung: Prüfen ob Start/Ziel frei sind
if cc_block.pointInCollision(start_block) or cc_block.pointInCollision(goal_block):
    print("WARNUNG: Benchmark B Start/Ziel kollidiert! Verschiebe Hindernis.")

#bench_block = CustomBenchmark("2. Obstacle Avoidance", cc_block, [start_block], [goal_block])
bench_block = Benchmark(
    name="2. Obstacle Avoidance",
    collisionChecker=cc_block,
    startList=[start_block],
    goalList=[goal_block],
    description="3-DOF planar arm must avoid a central block obstacle",
    level=2
)

# ZUSAMMENFASSUNG
benchmarks_1c = [bench_gap, bench_block]


# 4. VISUALISIERUNG BEIDER SZENARIEN 

# Config sicherstellen
config_arm = config.copy()
config_arm["initialRoadmapSize"] = 250 
config_arm["kNearest"] = 15 
config_arm["maxIterations"] = 60

print("Visualisiere Start- und Zielkonfigurationen...")

fig, axs = plt.subplots(1, 2, figsize=(14, 7))

for i, bench in enumerate(benchmarks_1c):
    ax = axs[i]
    ax.set_title(bench.name)
    ax.set_xlim(-4, 5)
    ax.set_ylim(-5, 5)
    ax.set_aspect('equal')

    # 1. Hindernisse zeichnen
    bench.collisionChecker.drawObstacles(ax, inWorkspace=True)

    # 2. Start (Blau)
    robot.move(bench.startList[0])
    planarRobotVisualize(robot, ax)
    # Beschriftung Start
    tip = robot.get_transforms()[-1]
    ax.text(tip[0], tip[1]+0.2, "START", color='blue', fontweight='bold')

    # 3. Ziel (Grün gestrichelt)
    robot.move(bench.goalList[0])
    # Manueller Plot für anderen Stil
    joints = robot.get_transforms()
    for j in range(1, len(joints)):
        ax.plot([joints[j-1][0], joints[j][0]], [joints[j-1][1], joints[j][1]], 
                color='green', linestyle='--', linewidth=2, alpha=0.7)
    
    # Beschriftung Ziel
    tip = joints[-1]
    ax.text(tip[0], tip[1]-0.5, "ZIEL", color='green', fontweight='bold')

plt.tight_layout()
plt.show()


# 5. STATISTISCHE AUSWERTUNG
print("\n--- Starte statistische Auswertung (Aufgabe 1c) ---")
# 15 Runs reichen meist für Manipulatoren, da Berechnung teuer ist

#Die Statistiken aufzustellen verbraucht sehr viel Zeit, daher auskommentiert lassen und nur für den finalen Vergleich nutzen.
#evaluate_and_plot(benchmarks_1c, config_arm, title_prefix="Aufgabe 1c", runs=15)

In [ ]:
# def interpolate_line(startPos, endPos, step_l):
#     # Hilfsfunktion für Interpolation im C-Space
#     steps = []
#     line = np.array(endPos) - np.array(startPos)
#     line_l = np.linalg.norm(line)
#     if line_l == 0: return [startPos]
    
#     step = line / line_l * step_l
#     n_steps = np.floor(line_l / step_l).astype(np.int32)
#     c_step = np.array(startPos)
#     for i in range(n_steps):
#         steps.append(copy.deepcopy(c_step))
#         c_step += step
#     if not (c_step == np.array(endPos)).all():
#         steps.append(np.array(endPos))
#     return steps

# # --- 2. BENCHMARKS DEFINIEREN (KORRIGIERT) ---

# benchmarks_1c = []

# # Roboter initialisieren (2 Gelenke, Standardlänge a=1.5 -> Gesamtreichweite = 3.0)
# r_easy = PlanarRobot(n_joints=2)
# r_hard = PlanarRobot(n_joints=2)

# # ---------------------------------------------------------
# # A) Leichtes Szenario (Original aus deinem Skript)
# # ---------------------------------------------------------
# obst_easy = dict()
# obst_easy["obs1"] = LineString([(-2, 0), (-0.8, 0)]).buffer(0.5)
# obst_easy["obs2"] = LineString([(2, 0), (2, 1)]).buffer(0.2)
# obst_easy["obs3"] = LineString([(-1, 2), (1, 2)]).buffer(0.1)

# cc_easy = KinChainCollisionChecker(r_easy, obst_easy, fk_resolution=0.1)

# start_easy = [2.0, 0.5]
# goal_easy  = [-2.0, -0.5]

# # Sicherheitscheck für Easy
# if cc_easy.pointInCollision(start_easy) or cc_easy.pointInCollision(goal_easy):
#     print("Warnung: Auch beim Easy-Benchmark kollidiert Start/Ziel. Versuche Ausweichwerte.")
#     start_easy = [1.5, 0.5] # Leicht angepasst

# benchmarks_1c.append(CustomBenchmark(
#     "Manipulator 2-DoF (Easy)", cc_easy, [start_easy], [goal_easy]
# ))


# # ---------------------------------------------------------
# # B) Schweres Szenario (Narrow Passage) - KORRIGIERT
# # ---------------------------------------------------------
# obst_hard = dict()

# # Wir bauen eine Wand bei x = 1.8 (innerhalb Reichweite 3.0)
# # Lücke ist zwischen y = -0.8 und y = 0.8
# obst_hard["wall_bottom"] = LineString([(1.8, -3.0), (1.8, -0.8)]).buffer(0.2)
# obst_hard["wall_top"]    = LineString([(1.8, 0.8),  (1.8, 3.0)]).buffer(0.2)

# cc_hard = KinChainCollisionChecker(r_hard, obst_hard, fk_resolution=0.1)

# # Start: Roboter zeigt gestreckt nach LINKS (x = -3.0). Sicher frei.
# start_hard = [np.pi, 0.0] 

# # Ziel: Roboter zeigt gestreckt nach RECHTS (x = +3.0). 
# # Er muss dafür durch die Lücke bei x=1.8 greifen.
# goal_hard  = [0.0, 0.0]  

# # Validierung (nur zur Sicherheit)
# if cc_hard.pointInCollision(start_hard):
#     print("FEHLER: Startpunkt Hard kollidiert immer noch!")
# if cc_hard.pointInCollision(goal_hard):
#     print("FEHLER: Zielpunkt Hard kollidiert immer noch!")

# benchmarks_1c.append(CustomBenchmark(
#     "Manipulator 2-DoF (Passage)", cc_hard, [start_hard], [goal_hard]
# ))


# # --- 3. AUSFÜHRUNG & VERGLEICH ---

# config_arm = config.copy()
# config_arm["initialRoadmapSize"] = 150 # Mehr Knoten für die Engstelle
# config_arm["kNearest"] = 15
# config_arm["maxIterations"] = 50

# # Visualisierungstest (Wir schauen uns das korrigierte Setup an)
# print("Visualisiere Szenario: Narrow Passage...")
# demo_bench = benchmarks_1c[1] # Hard
# fig, ax = plt.subplots(figsize=(8,8))
# ax.set_xlim(-4, 4)
# ax.set_ylim(-4, 4)

# # Hindernisse
# demo_bench.collisionChecker.drawObstacles(ax, inWorkspace=True)

# # Startkonfiguration (Blau)
# r_hard.move(demo_bench.startList[0])
# planarRobotVisualize(r_hard, ax)
# tip_start = r_hard.get_transforms()[-1]
# ax.text(tip_start[0], tip_start[1], " Start", fontsize=12, color='blue')

# # Zielkonfiguration (manuell Grün drüberzeichnen)
# r_hard.move(demo_bench.goalList[0])
# joints = r_hard.get_transforms()
# for i in range(1, len(joints)):
#     ax.plot([joints[i-1][0], joints[i][0]], [joints[i-1][1], joints[i][1]], color='green', linestyle='--')
# tip_goal = joints[-1]
# ax.text(tip_goal[0], tip_goal[1], " Ziel", fontsize=12, color='green')

# plt.title("Aufgabe 1c: Start (Links) vs Ziel (Rechts durch Lücke)")
# plt.show()

# # Statistische Auswertung

#Die Statistiken aufzustellen verbraucht sehr viel Zeit, daher auskommentiert lassen und nur für den finalen Vergleich nutzen.
# #evaluate_and_plot(benchmarks_1c, config_arm, title_prefix="Aufgabe 1c", runs=15)

import numpy as np
import copy
import matplotlib.pyplot as plt
from shapely.geometry import LineString
from IPBenchmark import Benchmark

# Hilfsfunktion global (muss hier stehen, damit die Klassen sie finden)
def interpolate_line(startPos, endPos, step_l):
    # Hilfsfunktion für Interpolation im C-Space
    steps = []
    line = np.array(endPos) - np.array(startPos)
    line_l = np.linalg.norm(line)
    if line_l == 0: return [startPos]
    
    step = line / line_l * step_l
    n_steps = np.floor(line_l / step_l).astype(np.int32)
    c_step = np.array(startPos)
    for i in range(n_steps):
        steps.append(copy.deepcopy(c_step))
        c_step += step
    if not (c_step == np.array(endPos)).all():
        steps.append(np.array(endPos))
    return steps

# SETUP (Roboter & CollisionChecker wie gehabt)

# Roboter initialisieren
r_easy = PlanarRobot(n_joints=2)
r_hard = PlanarRobot(n_joints=2)

# A) Leichtes Szenario
obst_easy = dict()
obst_easy["obs1"] = LineString([(-2, 0), (-0.8, 0)]).buffer(0.5)
obst_easy["obs2"] = LineString([(2, 0), (2, 1)]).buffer(0.2)
obst_easy["obs3"] = LineString([(-1, 2), (1, 2)]).buffer(0.1)

cc_easy = KinChainCollisionChecker(r_easy, obst_easy, fk_resolution=0.1)
start_easy = [2.0, 0.5]
goal_easy  = [-2.0, -0.5]

# Sicherheitscheck
if cc_easy.pointInCollision(start_easy) or cc_easy.pointInCollision(goal_easy):
    print("Warnung: Easy-Benchmark Start/Ziel kollidiert. Passe an.")
    start_easy = [1.5, 0.5]

#bench_easy = CustomBenchmark("Manipulator 2-DoF (Easy)", cc_easy, [start_easy], [goal_easy])
bench_easy = Benchmark(
    name="Manipulator 2-DoF (Easy)",
    collisionChecker=cc_easy,
    startList=[start_easy],
    goalList=[goal_easy],
    description="Easy obstacle avoidance with 2-DoF planar arm",
    level=1
)

# B) Schweres Szenario (Narrow Passage)
obst_hard = dict()
obst_hard["wall_bottom"] = LineString([(1.8, -3.0), (1.8, -0.8)]).buffer(0.2)
obst_hard["wall_top"]    = LineString([(1.8, 0.8),  (1.8, 3.0)]).buffer(0.2)

cc_hard = KinChainCollisionChecker(r_hard, obst_hard, fk_resolution=0.1)
start_hard = [np.pi, 0.0] 
goal_hard  = [0.0, 0.0]  

#bench_hard = CustomBenchmark("Manipulator 2-DoF (Passage)", cc_hard, [start_hard], [goal_hard])
bench_hard = Benchmark(
    name="Manipulator 2-DoF (Passage)",
    collisionChecker=cc_hard,
    startList=[start_hard],
    goalList=[goal_hard],
    description="2-DoF planar arm must pass a narrow vertical gap",
    level=3
)

# Liste aller Benchmarks
benchmarks_1c = [bench_easy, bench_hard]


# CONFIG 
config_arm = config.copy()
config_arm["initialRoadmapSize"] = 200
config_arm["kNearest"] = 15
config_arm["maxIterations"] = 50


# 1. STATISCHE VISUALISIERUNG (BEIDE SZENARIEN)
print("Visualisiere Start-/Zielkonfigurationen...")
fig, axs = plt.subplots(1, 2, figsize=(14, 7))

for i, bench in enumerate(benchmarks_1c):
    ax = axs[i]
    # Limits groß genug für beide Szenarien wählen
    ax.set_xlim(-4, 4)
    ax.set_ylim(-4, 4)
    ax.set_aspect('equal')
    ax.set_title(bench.name)

    # Hindernisse
    bench.collisionChecker.drawObstacles(ax, inWorkspace=True)

    # Start (Blau)
    robot_inst = bench.collisionChecker.kin_chain
    robot_inst.move(bench.startList[0])
    planarRobotVisualize(robot_inst, ax)
    tip = robot_inst.get_transforms()[-1]
    ax.text(tip[0], tip[1], " Start", color='blue', fontweight='bold')

    # Ziel (Grün gestrichelt - manueller Plot damit es sich unterscheidet)
    robot_inst.move(bench.goalList[0])
    joints = robot_inst.get_transforms()
    for j in range(1, len(joints)):
        ax.plot([joints[j-1][0], joints[j][0]], [joints[j-1][1], joints[j][1]], 
                color='green', linestyle='--', linewidth=2, alpha=0.7)
    tip = joints[-1]
    ax.text(tip[0], tip[1], " Ziel", color='green', fontweight='bold')

plt.tight_layout()
plt.show()


# 2. PLANUNG & ANIMATION (BEIDE SZENARIEN)
print("\n--- Starte Planung und Animation ---")
# HINWEIS: Dieser Teil funktioniert nur, wenn der Block mit 'animateSolution'
# und 'visualize_3d_projected' vorher ausgeführt wurde!

for bench in benchmarks_1c:
    print(f"\nPlane Pfad für: {bench.name}...")
    
    # Planer erstellen und Pfad suchen
    planner = LazyPRM(bench.collisionChecker)
    solution = planner.planPath(bench.startList, bench.goalList, config_arm)
    
    if solution:
        print(f"Pfad gefunden ({len(solution)} Knoten)! Erstelle Animation...")
        try:
            # Wir nutzen 'visualize_3d_projected' für den Graphen, 
            # da es auch mit 2D-Daten (projiziert dann halt nichts weg) funktioniert.
            animateSolution(planner, bench.collisionChecker, solution, visualize_3d_projected)
        except NameError:
            print("FEHLER: 'animateSolution' oder 'visualize_3d_projected' nicht gefunden.")
            print("Bitte stelle sicher, dass der Animations-Codeblock vorher ausgeführt wurde.")
    else:
        print("Kein Pfad gefunden. Versuche RoadmapSize oder maxIterations zu erhöhen.")

# Statistische Auswertung (auskommentiert lassen, wenn nur Visualisierung gewünscht)
# print("\n--- Starte Statistik ---")

#Die Statistiken aufzustellen verbraucht sehr viel Zeit, daher auskommentiert lassen und nur für den finalen Vergleich nutzen.
# evaluate_and_plot(benchmarks_1c, config_arm, title_prefix="Aufgabe 1c", runs=15)

In [ ]:
# INTERAKTIVE ANIMATION (Wie im Vorlesungs-Notebook)

import matplotlib.animation
from IPython.display import HTML, display

# Damit auch längere Pfade komplett animiert werden können (Limit erhöhen)
matplotlib.rcParams['animation.embed_limit'] = 512 

try:
    from IPVISLazyPRM import lazyPRMVisualize
except ImportError:
    print("Konnte Visualizer nicht laden. Bitte prüfen ob IPVISLazyPRM.py vorhanden ist.")

def animateSolution(planner, environment, solution, visualizer):
    """
    Erstellt eine HTML5-Animation der Roboterbewegung.
    Links: Arbeitsraum (Roboter bewegt sich).
    Rechts: C-Space (Graph und aktueller Zustand).
    """
    
    # 1. Pfad interpolieren (damit die Bewegung flüssig ist)
    solution_pos = [planner.graph.nodes[node]['pos'] for node in solution]
    
    # Wenn kein Pfad gefunden wurde, abbrechen
    if not solution_pos:
        print("Kein Pfad zur Animation gefunden!")
        return

    i_solution_pos = [solution_pos[0]]
    for i in range(1, len(solution_pos)):
        segment_s = solution_pos[i-1]
        segment_e = solution_pos[i]
        # Granularität: 0.05 für sehr weiche Bewegung
        i_solution_pos = i_solution_pos + interpolate_line(segment_s, segment_e, 0.05)[1:]
    
    frames = len(i_solution_pos)
    
    # 2. Plot Setup (2 Fenster nebeneinander)
    fig_anim = plt.figure(figsize=(14, 7))
    ax1 = fig_anim.add_subplot(1, 2, 1) # Arbeitsraum
    ax2 = fig_anim.add_subplot(1, 2, 2) # Graph/C-Space
    
    # Helper für das Zeichnen
    def update_frame(t):
        # -- Linkes Bild: Arbeitsraum --
        ax1.cla()
        ax1.set_xlim(-4, 4)
        ax1.set_ylim(-4, 4)
        ax1.set_title(f"Arbeitsraum (Step {t}/{frames})")
        
        # Hindernisse
        environment.drawObstacles(ax1, inWorkspace=True)
        
        # Roboter an aktueller Position
        current_conf = i_solution_pos[t]
        environment.kin_chain.move(current_conf)
        planarRobotVisualize(environment.kin_chain, ax1)
        
        # -- Rechtes Bild: Graph / C-Space --
        ax2.cla()
        ax2.set_title("Graph / Planung")
        # Graph zeichnen (nutzt den Visualizer aus den Imports)
        visualizer(planner, solution, ax=ax2)
        
        # Roten Punkt für aktuellen Zustand einzeichnen
        # (x = Gelenk 1, y = Gelenk 2 - nur bei 2D sinnvoll plotbar)
        if len(current_conf) >= 2:
            ax2.scatter(current_conf[0], current_conf[1], color='red', s=200, zorder=10, label='Current State')

    # 3. Animation erstellen
    print(f"Erstelle Animation mit {frames} Frames... (bitte warten)")
    ani = matplotlib.animation.FuncAnimation(fig_anim, update_frame, frames=frames, interval=50)
    
    # 4. Als HTML ausgeben (mit Start/Stop Controls)
    html = HTML(ani.to_jshtml())
    display(html)
    plt.close() # Verhindert doppelte Ausgabe des statischen Bildes

# ANWENDUNG AUF DAS SZENARIO

# Wir nehmen das 'Hard' Szenario aus der vorherigen Aufgabe
bench_anim = benchmarks_1c[1] # Index 1 war "Narrow Passage"
print(f"Berechne Pfad für Animation: {bench_anim.name}")

# Einmalig Planen für die Animation
anim_planner = LazyPRM(bench_anim.collisionChecker)
anim_sol = anim_planner.planPath(bench_anim.startList, bench_anim.goalList, config_arm)

if anim_sol:
    # Funktion aufrufen
    animateSolution(anim_planner, bench_anim.collisionChecker, anim_sol, lazyPRMVisualize)
else:
    print("Der Planer hat leider keinen Pfad gefunden, daher keine Animation.")

In [ ]:
# 3-DoF Manipulator (2 Szenarien & Animation)

import networkx as nx
import numpy as np
from IPBenchmark import Benchmark

# 1. SETUP 3-DoF ROBOTER
r_3dof = PlanarRobot(n_joints=3)
limits_3d = [[-np.pi, np.pi]] * 3 

# SZENARIO A: OBSTACLE AVOIDANCE (Leicht)
# Ein Block liegt im Weg, der Roboter muss "außenrum" schwenken.
obs_block = {
    "block": Polygon([(1.5, -1.0), (2.5, -1.0), (2.5, 1.0), (1.5, 1.0)])
}
cc_3dof_easy = KinChainCollisionChecker(r_3dof, obs_block, limits=limits_3d, fk_resolution=0.1)

# Start: Senkrecht nach OBEN
start_easy = [np.pi/2, 0.1, 0.1] 
# Ziel: Senkrecht nach UNTEN
goal_easy  = [-np.pi/2, 0.1, 0.1]

#bench_3dof_easy = CustomBenchmark("3-DoF (Obstacle Avoidance)", cc_3dof_easy, [start_easy], [goal_easy])
bench_3dof_easy = Benchmark(
    name="3-DoF (Obstacle Avoidance)",
    collisionChecker=cc_3dof_easy,
    startList=[start_easy],
    goalList=[goal_easy],
    description="3-DoF planar arm avoids a central block obstacle",
    level=1
)


# SZENARIO B: WALL GAP (Schwer)
# Wand bei x=2.5 mit Lücke.
obst_wall = dict()
obst_wall["wall_bottom"] = LineString([(2.5, -4.0), (2.5, -1.0)]).buffer(0.2)
obst_wall["wall_top"]    = LineString([(2.5, 1.0),  (2.5, 4.0)]).buffer(0.2)

cc_3dof_hard = KinChainCollisionChecker(r_3dof, obst_wall, limits=limits_3d, fk_resolution=0.1)

# Start: Links gestreckt
start_hard = [np.pi, 0.0, 0.0]
# Ziel: Rechts gestreckt (durch die Lücke)
goal_hard  = [0.0, 0.0, 0.0]

#bench_3dof_hard = CustomBenchmark("3-DoF (Wall Gap)", cc_3dof_hard, [start_hard], [goal_hard])
bench_3dof_hard = Benchmark(
    name="3-DoF (Wall Gap)",
    collisionChecker=cc_3dof_hard,
    startList=[start_hard],
    goalList=[goal_hard],
    description="3-DoF planar arm must pass through a narrow wall gap",
    level=3
)

# ZUSAMMENFASSUNG
benchmarks_3dof = [bench_3dof_easy, bench_3dof_hard]

# 2. CONFIG & STATISTIK
config_3dof = config.copy()
config_3dof["initialRoadmapSize"] = 350
config_3dof["kNearest"] = 15
config_3dof["maxIterations"] = 20 # Etwas Zeit geben

print("\n--- Starte 3-DoF Vergleich (Statistik) ---")
#Die Statistiken aufzustellen verbraucht sehr viel Zeit, daher auskommentiert lassen und nur für den finalen Vergleich nutzen.
# evaluate_and_plot(benchmarks_3dof, config_3dof, title_prefix="Zusatzaufgabe 3-DoF", runs=5)


# 3. INTERAKTIVE ANIMATION (BEIDE)

# Hilfsfunktion für die Projektion (damit Matplotlib bei 3D-Daten nicht abstürzt)
def visualize_3d_projected(planner, solution, ax=None):
    graph = planner.graph
    pos_3d = nx.get_node_attributes(graph, 'pos')
    # Wir nehmen nur die ersten beiden Gelenkwinkel für den Plot
    pos_2d = {node: coords[0:2] for node, coords in pos_3d.items()}
    
    nx.draw_networkx_nodes(graph, pos_2d, ax=ax, node_size=15, node_color='#1f77b4', alpha=0.6)
    nx.draw_networkx_edges(graph, pos_2d, ax=ax, alpha=0.2, edge_color='gray')
    
    if solution:
        path_edges = list(zip(solution, solution[1:]))
        nx.draw_networkx_edges(graph, pos_2d, ax=ax, edgelist=path_edges, edge_color='red', width=2)
        nx.draw_networkx_nodes(graph, pos_2d, ax=ax, nodelist=solution, node_size=30, node_color='red')
    
    if ax:
        ax.set_xlabel("Gelenk 1")
        ax.set_ylabel("Gelenk 2")


print("\n--- Starte Animation für beide 3-DoF Szenarien ---")

# SCHLEIFE ÜBER BEIDE BENCHMARKS
for bench in benchmarks_3dof:
    print(f"\nBerechne Animationspfad für: {bench.name}...")
    
    anim_planner = LazyPRM(bench.collisionChecker)
    anim_sol = anim_planner.planPath(bench.startList, bench.goalList, config_3dof)

    if anim_sol:
        print(f"Pfad gefunden! Erstelle Animation für {bench.name}...")
        # WICHTIG: animateSolution muss vorher definiert worden sein!
        animateSolution(anim_planner, bench.collisionChecker, anim_sol, visualize_3d_projected)
    else:
        print(f"Kein Pfad für {bench.name} gefunden (Versuche mehr Iterationen/Knoten).")

In [ ]:
#animateSolution(anim_planner_3, bench_3dof.collisionChecker, anim_sol_3, visualize_3d_projected)